## Importing necessary libraries

In [131]:
import zipfile
from pathlib import Path
import pandas as pd
from word2number import w2n
import pickle


## Data Preparation

In [132]:
data_path = Path('data')
Path.mkdir(data_path, exist_ok=True, parents=True)

In [133]:
with zipfile.ZipFile('data.zip', 'r') as file:
    file.extractall(data_path)

In [134]:
with open('data/invoices_new.pkl', 'rb') as f:
    voices = pickle.load(f)

In [135]:
with open('data/expired_invoices.txt', 'r') as f:
    expired = [line.strip() for line in f]

In [138]:
def convert_quantity(quantity):
    try:
        return int(quantity)
    except Exception:
        try:
            return w2n.word_to_num(quantity)
        except ValueError:
            return 0



# List to store the rows of the dataframe
rows = []

# Process each invoice
for idx, data in enumerate(voices):
    invoice_id = data['id']
    created_on = data['created_on']
    items = data.get('items', [])

    # Calculate total invoice amount
    total_invoice_amount = sum(
        item['item']['unit_price'] * convert_quantity(item['quantity']) 
        for item in items
    ) if items else 0

    # Process each item
    for item_entry in items:
        item = item_entry['item']
        quantity = convert_quantity(item_entry['quantity'])
        total_price = item['unit_price'] * quantity

        # Avoid division by zero if no items are present
        percentage_in_invoice = total_price / total_invoice_amount * 100
#         print(item['type'])
        row = {
            'invoice_id': invoice_id,
            'created_on': created_on,
            'invoiceitem_id': item['id'],
            'invoiceitem_name': item['name'],
            'type': item['type'],
            'unit_price': item['unit_price'],
            'total_price': total_price,
            'percentage_in_invoice': percentage_in_invoice,
            'is_expired': False  # Assuming you have a way to determine this
        }

        rows.append(row)
df = pd.DataFrame(rows)

df['created_on'] = pd.to_datetime(df['created_on'], errors='coerce')

# Create DataFrame
df['type'].replace({0:'Material',1:'Equipment',2:'Service', 3:'Other'}, inplace=True)



In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   invoice_id             492 non-null    object        
 1   created_on             486 non-null    datetime64[ns]
 2   invoiceitem_id         492 non-null    int64         
 3   invoiceitem_name       492 non-null    object        
 4   type                   492 non-null    object        
 5   unit_price             492 non-null    int64         
 6   total_price            492 non-null    int64         
 7   percentage_in_invoice  492 non-null    float64       
 8   is_expired             492 non-null    bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 31.4+ KB
